# Final Project

## Description

The telecom operator Interconnect would like to be able to forecast their churn of clients.

If it's discovered that a user is planning to leave, they will be offered promotional codes and special plan options.

Interconnect's marketing team has collected some of their clientele's personal data, including information about their plans and contracts.

# Initialization

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from catboost import Pool, cv
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
import random
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
import tensorflow as tf


import the files:

In [3]:
try:
    path='C:\\Users\\aviv\\Downloads\\final_provider\\'
    contract = pd.read_csv(path+'contract.csv', sep=",")
    personal = pd.read_csv(path+'personal.csv', sep=",")
    internet = pd.read_csv(path+'internet.csv', sep=",")
    phone = pd.read_csv(path+'phone.csv', sep=",")
except:
    contract = pd.read_csv("/datasets/contract.csv", sep=",")
    personal = pd.read_csv("/datasets/personal.csv", sep=",")
    internet = pd.read_csv("/datasets/internet.csv", sep=",")
    phone = pd.read_csv("/datasets/phone.csv", sep=",")
    
dfs=[contract, personal, internet, phone]
for i in dfs:
    display(i.head())

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65


,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No


,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


# Preprocessing

In [4]:
for i in dfs:
    i.columns= i.columns.str.lower()
    display(i.info())
    display(i.describe(include = object ))
    display(i.describe())
    
    print("null values:")

    
    data_mis = i.isnull().sum().to_frame('missing_values')
    data_mis['%'] = round(i.isnull().sum()/len(i),3)
    display(data_mis.sort_values(by='%', ascending=False))
    
    print("duplicated values: ")
    display(i.duplicated().sum())
    for column in i :
        if i[column].dtype=='object':
            before = i[column].nunique()
            i[column] = i[column].str.lower()
            if i[column].nunique() < before :
                print("unique values in:",column," change from:",before," to:",i[column].nunique())
            else:
                print("all object types are lowercased")
    
    
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7043 non-null   object 
 1   begindate         7043 non-null   object 
 2   enddate           7043 non-null   object 
 3   type              7043 non-null   object 
 4   paperlessbilling  7043 non-null   object 
 5   paymentmethod     7043 non-null   object 
 6   monthlycharges    7043 non-null   float64
 7   totalcharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB


None

,customerid,begindate,enddate,type,paperlessbilling,paymentmethod,totalcharges
count,7043,7043,7043,7043,7043,7043,7043
unique,7043,77,5,3,2,4,6531
top,7590-VHVEG,2014-02-01,No,Month-to-month,Yes,Electronic check,
freq,1,366,5174,3875,4171,2365,11


,monthlycharges
count,7043.000000
mean,64.761692
std,30.090047
min,18.250000
25%,35.500000
50%,70.350000
75%,89.850000
max,118.750000


null values:


...

null values:


,missing_values,%
customerid,0,0.0
multiplelines,0,0.0


duplicated values: 


0

all object types are lowercased
all object types are lowercased



no missing values, no duplicates.

## Work Plan
i intend to merge the tables for training, will null values show up?
i assume not all customers use all the services, so i'll have to fill these. after that, i'll try different models:

split the data to training, validation and test sets

train the models

evaluate by some evaluation metrics

and finally test the best model on the test set and conclude.

In [5]:
data = contract.merge(personal, on=['customerid'], how='outer') \
                .merge(internet, on=['customerid'], how='outer') \
                .merge(phone,    on=['customerid'], how='outer')
display(data.head())

,customerid,begindate,enddate,type,paperlessbilling,paymentmethod,monthlycharges,totalcharges,gender,seniorcitizen,partner,dependents,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,multiplelines
0,7590-vhveg,2020-01-01,no,month-to-month,yes,electronic check,29.85,29.85,female,0,yes,no,dsl,no,yes,no,no,no,no,NaN
1,5575-gnvde,2017-04-01,no,one year,no,mailed check,56.95,1889.5,male,0,no,no,dsl,yes,no,yes,no,no,no,no
2,3668-qpybk,2019-10-01,2019-12-01 00:00:00,month-to-month,yes,mailed check,53.85,108.15,male,0,no,no,dsl,yes,yes,no,no,no,no,no
3,7795-cfocw,2016-05-01,no,one year,no,bank transfer (automatic),42.30,1840.75,male,0,no,no,dsl,yes,no,yes,yes,no,no,NaN
4,9237-hqitu,2019-09-01,2019-11-01 00:00:00,month-to-month,yes,electronic check,70.70,151.65,female,0,no,no,fiber optic,no,no,no,no,no,no,no


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7043 non-null   object 
 1   begindate         7043 non-null   object 
 2   enddate           7043 non-null   object 
 3   type              7043 non-null   object 
 4   paperlessbilling  7043 non-null   object 
 5   paymentmethod     7043 non-null   object 
 6   monthlycharges    7043 non-null   float64
 7   totalcharges      7043 non-null   object 
 8   gender            7043 non-null   object 
 9   seniorcitizen     7043 non-null   int64  
 10  partner           7043 non-null   object 
 11  dependents        7043 non-null   object 
 12  internetservice   5517 non-null   object 
 13  onlinesecurity    5517 non-null   object 
 14  onlinebackup      5517 non-null   object 
 15  deviceprotection  5517 non-null   object 
 16  techsupport       5517 non-null   object 


we now have some NaN values, gotta fix those

In [7]:
data['internetservice'].fillna('no_internet', inplace=True)
data['multiplelines'].fillna('no_phone', inplace=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7043 non-null   object 
 1   begindate         7043 non-null   object 
 2   enddate           7043 non-null   object 
 3   type              7043 non-null   object 
 4   paperlessbilling  7043 non-null   object 
 5   paymentmethod     7043 non-null   object 
 6   monthlycharges    7043 non-null   float64
 7   totalcharges      7043 non-null   object 
 8   gender            7043 non-null   object 
 9   seniorcitizen     7043 non-null   int64  
 10  partner           7043 non-null   object 
 11  dependents        7043 non-null   object 
 12  internetservice   7043 non-null   object 
 13  onlinesecurity    5517 non-null   object 
 14  onlinebackup      5517 non-null   object 
 15  deviceprotection  5517 non-null   object 
 16  techsupport       5517 non-null   object 


In [9]:
for column in data.columns[12:]:

    data[column].fillna('no', inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7043 non-null   object 
 1   begindate         7043 non-null   object 
 2   enddate           7043 non-null   object 
 3   type              7043 non-null   object 
 4   paperlessbilling  7043 non-null   object 
 5   paymentmethod     7043 non-null   object 
 6   monthlycharges    7043 non-null   float64
 7   totalcharges      7043 non-null   object 
 8   gender            7043 non-null   object 
 9   seniorcitizen     7043 non-null   int64  
 10  partner           7043 non-null   object 
 11  dependents        7043 non-null   object 
 12  internetservice   7043 non-null   object 
 13  onlinesecurity    7043 non-null   object 
 14  onlinebackup      7043 non-null   object 
 15  deviceprotection  7043 non-null   object 
 16  techsupport       7043 non-null   object 


total charges is object type? should be int or float

In [10]:
data[pd.to_numeric(data.totalcharges, errors='coerce').isnull()]

,customerid,begindate,enddate,type,paperlessbilling,paymentmethod,monthlycharges,totalcharges,gender,seniorcitizen,partner,dependents,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,multiplelines
488,4472-lvygi,2020-02-01,no,two year,yes,bank transfer (automatic),52.55,,female,0,yes,yes,dsl,yes,no,yes,yes,yes,no,no_phone
753,3115-czmzd,2020-02-01,no,two year,no,mailed check,20.25,,male,0,no,yes,no_internet,no,no,no,no,no,no,no
936,5709-lvoeq,2020-02-01,no,two year,no,mailed check,80.85,,female,0,yes,yes,dsl,yes,yes,yes,no,yes,yes,no
1082,4367-nuyao,2020-02-01,no,two year,no,mailed check,25.75,,male,0,yes,yes,no_internet,no,no,no,no,no,no,yes
1340,1371-dwpaz,2020-02-01,no,two year,no,credit card (automatic),56.05,,female,0,yes,yes,dsl,yes,yes,yes,yes,yes,no,no_phone
3331,7644-omvmy,2020-02-01,no,two year,no,mailed check,19.85,,male,0,yes,yes,no_internet,no,no,no,no,no,no,no
3826,3213-vvolg,2020-02-01,no,two year,no,mailed check,25.35,,male,0,yes,yes,no_internet,no,no,no,no,no,no,yes
4380,2520-sgtta,2020-02-01,no,two year,no,mailed check,20.00,,female,0,yes,yes,no_internet,no,no,no,no,no,no,no
5218,2923-arzlg,2020-02-01,no,one year,yes,mailed check,19.70,,male,0,yes,yes,no_internet,no,no,no,no,no,no,no
6670,4075-wkniu,2020-02-01,no,two year,no,mailed check,73.35,,female,0,yes,yes,dsl,no,yes,yes,yes,yes,no,yes


begin date is all the same, so the company hadn't received payment from these, let's fill these with zeroes

In [11]:
data.totalcharges = pd.to_numeric(data.totalcharges, errors='coerce').fillna(0.0)

In [12]:
data.totalcharges.isnull().sum()

0

In [13]:
data['totalcharges'] = data['totalcharges'].astype('float')

converting datetime columns to datetime type

In [14]:
data['enddate'] = pd.to_datetime(data['enddate'], format='%Y-%m-%d', errors='coerce')
data['begindate'] = pd.to_datetime(data['begindate'], format='%Y-%m-%d', errors='coerce')

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerid        7043 non-null   object        
 1   begindate         7043 non-null   datetime64[ns]
 2   enddate           1869 non-null   datetime64[ns]
 3   type              7043 non-null   object        
 4   paperlessbilling  7043 non-null   object        
 5   paymentmethod     7043 non-null   object        
 6   monthlycharges    7043 non-null   float64       
 7   totalcharges      7043 non-null   float64       
 8   gender            7043 non-null   object        
 9   seniorcitizen     7043 non-null   int64         
 10  partner           7043 non-null   object        
 11  dependents        7043 non-null   object        
 12  internetservice   7043 non-null   object        
 13  onlinesecurity    7043 non-null   object        
 14  onlinebackup      7043 n

## Preparation for training

our target is the exit date -> churn. add columns for duration and churn.

In [16]:
data['duration'] = data['enddate'] - data['begindate']
data['duration'] = data['duration'].fillna(pd.to_datetime('2020-02-01') - data['begindate'])
data['duration'] = data['duration'].dt.days

In [17]:
data['churn'] = (~data['enddate'].isna()).astype('int')

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerid        7043 non-null   object        
 1   begindate         7043 non-null   datetime64[ns]
 2   enddate           1869 non-null   datetime64[ns]
 3   type              7043 non-null   object        
 4   paperlessbilling  7043 non-null   object        
 5   paymentmethod     7043 non-null   object        
 6   monthlycharges    7043 non-null   float64       
 7   totalcharges      7043 non-null   float64       
 8   gender            7043 non-null   object        
 9   seniorcitizen     7043 non-null   int64         
 10  partner           7043 non-null   object        
 11  dependents        7043 non-null   object        
 12  internetservice   7043 non-null   object        
 13  onlinesecurity    7043 non-null   object        
 14  onlinebackup      7043 n

In [19]:
data.churn.value_counts()

0    5174
1    1869
Name: churn, dtype: int64

let's drop columns that are irrelevant for training:

In [20]:
df=data.copy()
df = df.drop(['customerid','begindate','enddate'], axis=1)

encode the categorical features:

In [21]:
cats = list(df.select_dtypes(include=['object']).columns)

encoder = OrdinalEncoder() 
encoder.fit(df[cats]) 

df[cats] = encoder.transform(df[cats]).astype('int64')

## splitting to training, validation and test sets

In [22]:
train, test = train_test_split(df, test_size = 0.3, stratify = df['churn'], random_state=12345)
valid, test = train_test_split(test, test_size = 0.5, stratify = test['churn'], random_state=12345)

In [23]:
X_train = train.drop('churn', axis = 1)
X_valid = valid.drop('churn', axis = 1)
X_test  = test.drop('churn', axis = 1)

y_train = train['churn']
y_valid = valid['churn']
y_test  = test['churn']

In [24]:
print('dataFrame shape:',df.shape)
print('X_train shape:',X_train.shape) 
print('y_train shape:',y_train.shape)   
print('X_valid shape:',X_valid.shape) 
print('y_train shape:',y_valid.shape)    

print('X_test shape:',X_test.shape)  
print('y_test shape:',y_test.shape) 

dataFrame shape: (7043, 19)
X_train shape: (4930, 18)
y_train shape: (4930,)
X_valid shape: (1056, 18)
y_train shape: (1056,)
X_test shape: (1057, 18)
y_test shape: (1057,)


normalizing

In [25]:
numeric = ['monthlycharges', 'duration']
scaler = StandardScaler()
scaler.fit(X_train[numeric])
for i in [X_train, X_valid, X_test]:
    i[numeric] = scaler.transform(i[numeric])


# Modeling

## Sanity check

In [54]:

y_sanity_prob = [random.random() for i in range(y_train.shape[0])]

y_sanity_pred = [0 if i < 0.5 else 1 for i in y_sanity_prob]

def eval(y,pred,prob):
    
    acc=np.round(accuracy_score(y, pred),2)
    recall=np.round(recall_score(y, pred),2)
    precision=np.round(precision_score(y, pred),2)
    f1=np.round(f1_score(y, pred),2)
    auc_roc = roc_auc_score(y, prob)
    print('accuracy:', acc)
    print('recall:', recall)
    print('precision:', precision)
    print('f1:', f1)
    print('AUC-ROC:', auc_roc)

eval(y_train,y_sanity_pred, y_sanity_prob)



accuracy: 0.5
recall: 0.51
precision: 0.27
f1: 0.35
AUC-ROC: 0.5032406867984809


looks great, let's proceed to the models

## Random Forest

In [55]:
def trainer_eval(model,X_train, y_train, X_valid, y_valid):
    model.fit(X_train, y_train)

    y_pred = model.predict(X_valid)
    y_prob = model.predict_proba(X_valid)
    eval(y_valid, y_pred, y_prob[:,1])

In [92]:
model = RandomForestClassifier(random_state=12345,n_estimators = 100 )

best_model = RandomForestClassifier(random_state=12345,n_estimators = 100 )

trainer_eval(model, X_train, y_train, X_valid, y_valid)

accuracy: 0.82
recall: 0.52
precision: 0.73
f1: 0.61
AUC-ROC: 0.8553042157584685


In [98]:
model = RandomForestClassifier(random_state=12345,n_estimators = 50 )

trainer_eval(model, X_train, y_train, X_valid, y_valid)

accuracy: 0.82
recall: 0.51
precision: 0.71
f1: 0.6
AUC-ROC: 0.8531157952871871




*After fixing the AUC ROC calculation, this model doesn't seem so bad after all.

this is the best model

## Tensorflow Neural network

In [81]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

adam = tf.keras.optimizers.Adam(learning_rate=0.0002)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics='accuracy')

model.fit(X_train, y_train, epochs=200, batch_size=32)

Epoch 1/200
155/155 [==============================] - 5s 11ms/step - loss: 4.3040 - accuracy: 0.6836
Epoch 2/200
155/155 [==============================] - 1s 7ms/step - loss: 0.6488 - accuracy: 0.7375
Epoch 3/200
155/155 [==============================] - 1s 6ms/step - loss: 0.8097 - accuracy: 0.7308
Epoch 4/200
155/155 [==============================] - 1s 6ms/step - loss: 0.6505 - accuracy: 0.7483
Epoch 5/200
155/155 [==============================] - 1s 7ms/step - loss: 0.8259 - accuracy: 0.7381
Epoch 6/200
155/155 [==============================] - 1s 7ms/step - loss: 0.8423 - accuracy: 0.7456
Epoch 7/200
155/155 [==============================] - 1s 7ms/step - loss: 0.6055 - accuracy: 0.7574
Epoch 8/200
155/155 [==============================] - 1s 5ms/step - loss: 0.6391 - accuracy: 0.7525
Epoch 9/200
155/155 [==============================] - 1s 5ms/step - loss: 0.6271 - accuracy: 0.7590
Epoch 10/200
155/155 [==============================] - 1s 5ms/step - loss: 0.6318 - accur

In [85]:
y_prob = model.predict(X_valid)
y_pred = [0 if i < 0.5 else 1 for i in y_prob]

eval(y_valid,y_pred, y_prob)

33/33 [==============================] - 0s 6ms/step
accuracy: 0.79
recall: 0.43
precision: 0.67
f1: 0.53
AUC-ROC: 0.8202503681885124


decent model let's check if we can do better

## Linear Regression

In [96]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [97]:
y_prob = model.predict(X_valid)
y_pred = [0 if i < 0.5 else 1 for i in y_prob]

eval(y_valid,y_pred, y_prob)

accuracy: 0.79
recall: 0.48
precision: 0.63
f1: 0.54
AUC-ROC: 0.8315077319587628


## Logistic Regression

In [88]:
model = LogisticRegression()
model.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [89]:
y_prob = model.predict(X_valid)
y_pred = [0 if i < 0.5 else 1 for i in y_prob]

eval(y_valid,y_pred, y_prob)

accuracy: 0.79
recall: 0.52
precision: 0.62
f1: 0.56
AUC-ROC: 0.7015832106038291


## CatBoost

In [90]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.05, depth=4)
model.fit(X_train, y_train)

# iterations':[800,900,1000]
# learning_rate':[0.19,0.2,0.21],
# depth': range(2, 4, 1),

0:	learn: 0.6620049	total: 26.1ms	remaining: 26.1s
1:	learn: 0.6348230	total: 43.5ms	remaining: 21.7s
2:	learn: 0.6103812	total: 66.9ms	remaining: 22.2s
3:	learn: 0.5871665	total: 93.6ms	remaining: 23.3s
4:	learn: 0.5684779	total: 116ms	remaining: 23.2s
5:	learn: 0.5531055	total: 141ms	remaining: 23.3s
6:	learn: 0.5375024	total: 159ms	remaining: 22.6s
7:	learn: 0.5230933	total: 182ms	remaining: 22.5s
8:	learn: 0.5130701	total: 205ms	remaining: 22.5s
9:	learn: 0.5025412	total: 227ms	remaining: 22.4s
10:	learn: 0.4912413	total: 244ms	remaining: 21.9s
11:	learn: 0.4848464	total: 262ms	remaining: 21.6s
12:	learn: 0.4760479	total: 285ms	remaining: 21.7s
13:	learn: 0.4696629	total: 302ms	remaining: 21.3s
14:	learn: 0.4650499	total: 317ms	remaining: 20.8s
15:	learn: 0.4599994	total: 331ms	remaining: 20.4s
16:	learn: 0.4546468	total: 344ms	remaining: 19.9s
17:	learn: 0.4496256	total: 358ms	remaining: 19.5s
18:	learn: 0.4437767	total: 375ms	remaining: 19.4s
19:	learn: 0.4407689	total: 414ms	rem

In [91]:
y_prob = model.predict(X_valid)
y_pred = [0 if i < 0.5 else 1 for i in y_prob]

eval(y_valid,y_pred, y_prob)

accuracy: 0.88
recall: 0.67
precision: 0.83
f1: 0.74
AUC-ROC: 0.8112297496318113


tweaked some parameters until i got a decent score. 

# Using the best model on the test set - RandomForestClassifier

In [101]:
best_model.fit(X_train, y_train)
y_prob = best_model.predict_proba(X_test)
y_pred = best_model.predict(X_test)

eval(y_test,y_pred, y_prob[:,1])

accuracy: 0.82
recall: 0.53
precision: 0.72
f1: 0.61
AUC-ROC: 0.8373261914370621


our model did well enough.

# Conclusion

after preprocessing, encoding, normalizing and trying a few models, we reached reached our goal with Random forest, and AUC_ROC of at least 0.75 (it's about 0.83)

# Solution Report

- What steps of the plan were performed and what steps were skipped (explain why)?
    * All the planned steps were performed, although i saw fit to add some steps like encoding, normalizing and adding a target column
    

- What difficulties did you encounter and how did you manage to solve them?
    * I had a problem calculating the AUC-ROC values as i was using binary predictions instead of probabilities. started using predict_proba to get continuous predictions, and then convert those to binary to calculate accuracy.


- What were some of the key steps to solving the task?

    * Some of the key steps to solving the tasks were normalizing the numerical values, encoding the categorical values and  splitting the dataset to training, validation and test sets.

- What is you final model and what quality score does it have?
    * My final model is RandomForestClassifier with n_estimators=100 and its AUC-ROC is 83%
